# Validation of the Mixing Matrix with spatial variability

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from fgbuster.observation_helpers import get_instrument, standardize_instrument, get_observation
from micmac import *

In [2]:
# Choose and get instrument from public database
instr_name = 'SO_SAT'    # LiteBIRD  # SO_SAT
instr_ = get_instrument(instr_name)
instrument = standardize_instrument(instr_)

In [3]:
# General parameters (in a Gibbs sampling run coming from the parameter file)
NSIDE = 64
ncomp = 3
n_fgs_comp = 2
pos_special_freqs = np.array([0, -1])

### Spv configuration

In [4]:
# Num betas
n_betas = (ncomp-1)*(len(instrument.frequency)-ncomp+1)

In [5]:
# Read the spv configuration and print the spv tree
yaml_file_path = 'params_spv_SAT.yaml'   # ''  # 'params_spv_SAT.yaml'  # 'params_spv_LB.yaml'
root_tree = tree_spv_config(yaml_file_path, n_betas, n_fgs_comp, print_tree=True)

<_io.TextIOWrapper name='params_spv_SAT.yaml' mode='r' encoding='UTF-8'>
count_b: 8
n_betas:  8

>>> Tree of spv config as passed by the User:
root
  nside_spv
    default: [1]
    f1
      default: [1]
      b0
        default: [1]
      b1
        default: None
      b2
        default: None
      b3
        default: None
    f2
      default: None
      b0
        default: None
      b1
        default: None
      b2
        default: None
      b3
        default: None

>>> Tree of spv config after filling the missing values:
root
  nside_spv
    default: [1]
    f1
      default: [1]
      b0
        default: [1]
      b1
        default: [1]
      b2
        default: [1]
      b3
        default: [1]
    f2
      default: [1]
      b0
        default: [1]
      b1
        default: [1]
      b2
        default: [1]
      b3
        default: [1]


In [6]:
# Get list of the nodes betas
nodes_b = get_nodes_b(root_tree)
print("List of all the betas (first all the ones for component 1 then all the ones for compoents 2):\n", nodes_b)

List of all the betas (first all the ones for component 1 then all the ones for compoents 2):
 [Node('/root/nside_spv/f1/b0'), Node('/root/nside_spv/f1/b1'), Node('/root/nside_spv/f1/b2'), Node('/root/nside_spv/f1/b3'), Node('/root/nside_spv/f2/b0'), Node('/root/nside_spv/f2/b1'), Node('/root/nside_spv/f2/b2'), Node('/root/nside_spv/f2/b3')]


### Initialization Mixing Matrix

In [7]:
initB = InitMixingMatrix(freqs=instrument.frequency, ncomp=ncomp, pos_special_freqs=pos_special_freqs, spv_nodes_b=nodes_b)
init_params = initB.init_params()
print("N params:", init_params.shape)
print("Params initial values (given in the order: all params for b1, all params for b2, ...):\n", init_params)


N params: (96,)
Params initial values (given in the order: all params for b1, all params for b2, ...):
 [0.33858815 0.33858815 0.33858815 0.33858815 0.33858815 0.33858815
 0.33858815 0.33858815 0.33858815 0.33858815 0.33858815 0.33858815
 0.02963068 0.02963068 0.02963068 0.02963068 0.02963068 0.02963068
 0.02963068 0.02963068 0.02963068 0.02963068 0.02963068 0.02963068
 0.00998213 0.00998213 0.00998213 0.00998213 0.00998213 0.00998213
 0.00998213 0.00998213 0.00998213 0.00998213 0.00998213 0.00998213
 0.00328725 0.00328725 0.00328725 0.00328725 0.00328725 0.00328725
 0.00328725 0.00328725 0.00328725 0.00328725 0.00328725 0.00328725
 0.00979233 0.00979233 0.00979233 0.00979233 0.00979233 0.00979233
 0.00979233 0.00979233 0.00979233 0.00979233 0.00979233 0.00979233
 0.05147511 0.05147511 0.05147511 0.05147511 0.05147511 0.05147511
 0.05147511 0.05147511 0.05147511 0.05147511 0.05147511 0.05147511
 0.12913911 0.12913911 0.12913911 0.12913911 0.12913911 0.12913911
 0.12913911 0.12913911 0.

### Mixing Matrix

Test preliminary functions

In [8]:
print(get_len_params(spv_nodes_b=nodes_b))

96


Test Mixing Matrix

In [9]:
B = MixingMatrix(instrument.frequency, ncomp, nodes_b, NSIDE, init_params, pos_special_freqs)

In [10]:
params = B.params

In [11]:
params_long = B.get_params_long(jax_use=True)
print(params_long.shape)
print(params_long)

(4, 2, 49152)
[[[0.33858815 0.33858815 0.33858815 ... 0.33858815 0.33858815 0.33858815]
  [0.00979233 0.00979233 0.00979233 ... 0.00979233 0.00979233 0.00979233]]

 [[0.02963068 0.02963068 0.02963068 ... 0.02963068 0.02963068 0.02963068]
  [0.05147511 0.05147511 0.05147511 ... 0.05147511 0.05147511 0.05147511]]

 [[0.00998213 0.00998213 0.00998213 ... 0.00998213 0.00998213 0.00998213]
  [0.12913911 0.12913911 0.12913911 ... 0.12913911 0.12913911 0.12913911]]

 [[0.00328725 0.00328725 0.00328725 ... 0.00328725 0.00328725 0.00328725]
  [0.44076761 0.44076761 0.44076761 ... 0.44076761 0.44076761 0.44076761]]]


In [12]:
B_fgs = B.get_B_fgs()

In [13]:
B_fgs[:,:,0]

array([[1.        , 0.        ],
       [0.33858815, 0.00979233],
       [0.02963068, 0.05147511],
       [0.00998213, 0.12913911],
       [0.00328725, 0.44076761],
       [0.        , 1.        ]])

In [14]:
B.get_B_cmb().shape

(6, 1, 49152)

In [15]:
B.get_B()[:,:,0]

array([[1.        , 1.        , 0.        ],
       [1.        , 0.33858815, 0.00979233],
       [1.        , 0.02963068, 0.05147511],
       [1.        , 0.00998213, 0.12913911],
       [1.        , 0.00328725, 0.44076761],
       [1.        , 0.        , 1.        ]])

In [24]:
new_params = B.params
new_params[0] += 10
new_params[1] += 20
new_params[50] += 20
new_params

array([2.03385881e+01, 4.03385881e+01, 3.38588146e-01, 3.38588146e-01,
       3.38588146e-01, 3.38588146e-01, 3.38588146e-01, 3.38588146e-01,
       3.38588146e-01, 3.38588146e-01, 3.38588146e-01, 3.38588146e-01,
       2.96306809e-02, 2.96306809e-02, 2.96306809e-02, 2.96306809e-02,
       2.96306809e-02, 2.96306809e-02, 2.96306809e-02, 2.96306809e-02,
       2.96306809e-02, 2.96306809e-02, 2.96306809e-02, 2.96306809e-02,
       9.98212728e-03, 9.98212728e-03, 9.98212728e-03, 9.98212728e-03,
       9.98212728e-03, 9.98212728e-03, 9.98212728e-03, 9.98212728e-03,
       9.98212728e-03, 9.98212728e-03, 9.98212728e-03, 9.98212728e-03,
       3.28725430e-03, 3.28725430e-03, 3.28725430e-03, 3.28725430e-03,
       3.28725430e-03, 3.28725430e-03, 3.28725430e-03, 3.28725430e-03,
       3.28725430e-03, 3.28725430e-03, 3.28725430e-03, 3.28725430e-03,
       9.79233500e-03, 9.79233500e-03, 2.00097923e+01, 9.79233500e-03,
       9.79233500e-03, 9.79233500e-03, 9.79233500e-03, 9.79233500e-03,
      

In [25]:
B.update_params(new_params)
print(B.get_params_long(jax_use=True))


[[[2.03385881e+01 4.03385881e+01 3.38588146e-01 ... 3.38588146e-01
   3.38588146e-01 3.38588146e-01]
  [9.79233500e-03 9.79233500e-03 2.00097923e+01 ... 9.79233500e-03
   9.79233500e-03 9.79233500e-03]]

 [[2.96306809e-02 2.96306809e-02 2.96306809e-02 ... 2.96306809e-02
   2.96306809e-02 2.96306809e-02]
  [5.14751134e-02 5.14751134e-02 5.14751134e-02 ... 5.14751134e-02
   5.14751134e-02 5.14751134e-02]]

 [[9.98212728e-03 9.98212728e-03 9.98212728e-03 ... 9.98212728e-03
   9.98212728e-03 9.98212728e-03]
  [1.29139113e-01 1.29139113e-01 1.29139113e-01 ... 1.29139113e-01
   1.29139113e-01 1.29139113e-01]]

 [[3.28725430e-03 3.28725430e-03 3.28725430e-03 ... 3.28725430e-03
   3.28725430e-03 3.28725430e-03]
  [4.40767614e-01 4.40767614e-01 4.40767614e-01 ... 4.40767614e-01
   4.40767614e-01 4.40767614e-01]]]


In [26]:
B.indexes_b

array([[ 0., 48.],
       [12., 60.],
       [24., 72.],
       [36., 84.]])

In [27]:
B.indexes_b.ravel(order='F')

array([ 0., 12., 24., 36., 48., 60., 72., 84.])